In [2]:
# Script Name: Option_1 (Arbitrage Bot with Multi-threaded Scanning and Full Cycle Management)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import json
import os
import concurrent.futures

# API Keys (Bybit and Bitget only for this trial)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# Connect to APIs
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})


In [ ]:
# Settings
SPREAD_THRESHOLD = 0.7 / 100
MIN_VOLUME = 50000
CHECK_INTERVAL = 1
TRADE_USD_SIZE = 30
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit, 'Bitget': bitget}
futures_markets_cache = {}
TRADE_FILE = 'open_trades.json'

# Load/Save Trades
def load_open_trades():
    if os.path.exists(TRADE_FILE):
        with open(TRADE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_open_trades(trades):
    with open(TRADE_FILE, 'w') as f:
        json.dump(trades, f, indent=2)

open_trades = load_open_trades()

def list_open_trades():
    if not open_trades:
        print("📭 No open trades.")
        return
    print("\n📦 Open Trades:")
    for i, (tid, t) in enumerate(open_trades.items()):
        print(f"[{i}] {tid} | {t['symbol']} on {t['exchange']} | {t['amount']} units @ Spot {t['spot_price']} / Futures {t['futures_price']}")

def ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%H:%M:%S')
    prompt = (f"\n⏰ {timestamp} | ⚠️ Arbitrage Signal on {exchange_name} - {symbol}\n"
              f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}\n"
              f"Do you want to execute this trade for {exchange_name} {symbol}? (yes/no): ")
    choice = input(prompt).strip().lower()
    return choice == 'yes'

def get_best_bid_ask(exchange, symbol):
    order_book = exchange.fetch_order_book(symbol)
    best_bid = order_book['bids'][0][0] if order_book['bids'] else None
    best_ask = order_book['asks'][0][0] if order_book['asks'] else None
    return best_bid, best_ask

def execute_trade(exchange_name, symbol, spot_price, futures_price):
    try:
        exchange = exchanges[exchange_name]
        base_asset = symbol.split('/')[0]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)
        amount = TRADE_USD_SIZE / spot_price
        best_bid, _ = get_best_bid_ask(exchange, symbol)
        order = exchange.create_order(
            symbol=symbol, type='limit', side='buy', amount=amount, price=best_bid
        )
        max_wait, wait_time, filled = 15, 0, 0
        while wait_time < max_wait:
            current_order = exchange.fetch_order(order['id'], symbol)
            filled = current_order['filled']
            if filled > 0:
                break
            current_price = get_best_bid_ask(exchange, symbol)[0]
            if current_order['price'] != current_price:
                exchange.cancel_order(order['id'], symbol)
                order = exchange.create_order(
                    symbol=symbol, type='limit', side='buy', amount=amount, price=current_price
                )
            time.sleep(1)
            wait_time += 1
        if filled == 0:
            exchange.cancel_order(order['id'], symbol)
            print("❌ Spot order did not fill. Aborting futures leg.")
            return
        # Set up futures exchange
        if exchange_name == 'Bybit':
            futures_exchange = ccxt.bybit({
                'apiKey': BYBIT_API_KEY,
                'secret': BYBIT_API_SECRET,
                'enableRateLimit': True,
                'options': {'defaultType': 'future'}
            })
        else:
            futures_exchange = ccxt.bitget({
                'apiKey': BITGET_API_KEY,
                'secret': BITGET_API_SECRET,
                'password': BITGET_API_PASSWORD,
                'enableRateLimit': True,
                'options': {'defaultType': 'swap'}
            })
        futures_exchange.set_leverage(1, futures_symbol)
        futures_exchange.create_order(
            symbol=futures_symbol, type='market', side='sell', amount=filled
        )
        trade_id = f"{symbol.replace('/', '')}_{int(time.time())}"
        open_trades[trade_id] = {
            "symbol": symbol,
            "exchange": exchange_name,
            "amount": filled,
            "spot_price": spot_price,
            "futures_price": futures_price,
            "timestamp": datetime.datetime.now().isoformat()
        }
        save_open_trades(open_trades)
        print(f"✅ Trade saved as {trade_id}")
    except Exception as e:
        print(f"❌ Trade Error: {e}")

def close_trade(trade_id):
    try:
        trade = open_trades[trade_id]
        symbol = trade['symbol']
        amount = trade['amount']
        exchange_name = trade['exchange']
        if exchange_name == 'Bybit':
            spot_ex = ccxt.bybit({
                'apiKey': BYBIT_API_KEY,
                'secret': BYBIT_API_SECRET,
                'enableRateLimit': True,
                'options': {'defaultType': 'spot'}
            })
            fut_ex = ccxt.bybit({
                'apiKey': BYBIT_API_KEY,
                'secret': BYBIT_API_SECRET,
                'enableRateLimit': True,
                'options': {'defaultType': 'future'}
            })
        else:
            spot_ex = ccxt.bitget({
                'apiKey': BITGET_API_KEY,
                'secret': BITGET_API_SECRET,
                'password': BITGET_API_PASSWORD,
                'enableRateLimit': True,
                'options': {'defaultType': 'spot'}
            })
            fut_ex = ccxt.bitget({
                'apiKey': BITGET_API_KEY,
                'secret': BITGET_API_SECRET,
                'password': BITGET_API_PASSWORD,
                'enableRateLimit': True,
                'options': {'defaultType': 'swap'}
            })
        base = symbol.split('/')[0]
        futures_symbol = find_futures_symbol(symbol, futures_markets_cache.get(exchange_name, []))
        spot_ex.create_order(symbol=symbol, type='market', side='sell', amount=amount)
        fut_ex.set_leverage(1, futures_symbol)
        fut_ex.create_order(symbol=futures_symbol, type='market', side='buy', amount=amount)
        del open_trades[trade_id]
        save_open_trades(open_trades)
        print(f"✅ Trade {trade_id} closed.")
    except Exception as e:
        print(f"❌ Error closing trade: {e}")

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except:
        return None

def fetch_prices_concurrently(exchange, symbols):
    prices = {}
    def fetch(symbol):
        prices[symbol] = get_price(exchange, symbol)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(fetch, symbols)
    return prices

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol]
        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    volume = exchange.fetch_ticker(symbol).get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except:
                    continue
        return tradable_tokens
    except:
        return []

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_prices, futures_prices):
    spot_price = spot_prices.get(symbol)
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbol = find_futures_symbol(symbol, futures_list)
    futures_price = futures_prices.get(futures_symbol)
    if not spot_price or not futures_price:
        return
    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)
    if net_profit >= SPREAD_THRESHOLD:
        if ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
            execute_trade(exchange_name, symbol, spot_price, futures_price)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    spot_symbols = tokens
    futures_symbols = [find_futures_symbol(t, futures_list) for t in tokens]
    symbol_map = {spot: fut for spot, fut in zip(spot_symbols, futures_symbols) if fut}
    spot_prices = fetch_prices_concurrently(exchange, list(symbol_map.keys()))
    futures_prices = fetch_prices_concurrently(exchange, list(symbol_map.values()))
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_intra_exchange_arbitrage, exchange, exchange_name, symbol, spot_prices, futures_prices) for symbol in symbol_map.keys()]
        concurrent.futures.wait(futures)

def main():
    while True:
        try:
            print("\n--- Arbitrage Bot ---")
            list_open_trades()
            choice = input("Enter trade ID to close or press Enter to scan: ").strip()
            if choice and choice in open_trades:
                close_trade(choice)
            else:
                for exchange_name, exchange in exchanges.items():
                    tokens = get_tradable_tokens(exchange, exchange_name)
                    process_tokens(exchange, exchange_name, tokens)
                time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(5)

if __name__ == "__main__":
    main()


--- Arbitrage Bot ---
📭 No open trades.


Enter trade ID to close or press Enter to scan:  
